In [3]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from utils.clt_repetidos import *
from datetime import datetime
import cx_Oracle
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from utils.clt_repetidos import * 
from ydata_profiling import ProfileReport
import dask.dataframe as dd
import gc
import matplotlib.pyplot as plt

# Get the data and create the output directory (Extraction)

In [1]:
path_dir_data = "/home/ale1726/proyects/datalake/clientes/data/productos/SIMS/data"

In [4]:
# Obtenemos la fecha actual para crear un directorio
# donde se guardaran los archivos de salida
date_now = datetime.now().strftime("%d_%m_%Y") 
path_dir_today =  os.path.join(path_dir_data, date_now)

# Creamos el directorio si no existe
os.makedirs(path_dir_today, exist_ok=True)

In [5]:
query_productos_sims = """ 
WITH CLIENTES_ACTIVOS AS (
    SELECT CLAVE, NOMBRE
    FROM SIMS.SIMS_CONTRAPARTES 
    WHERE NOMBRE NOT LIKE '%INACTI%' AND NOMBRE NOT LIKE '%APLICA%'
), MOVIMIENTO AS (
    SELECT SM.*, CAT.DESCRIPCION MOVIMIENTO
    FROM SIMS.SIMS_MOVIMIENTOS SM
    LEFT JOIN SIMS.SIMS_TIPOS_MOVTOS CAT ON CAT.CLAVE = SM.TIPO_MOVTO
),  MOVIMIENTO_C2 AS (
    SELECT SM.*, CAT.DESCRIPCION PORTAFOLIO
    FROM MOVIMIENTO SM
    LEFT JOIN SIMS.SIMS_TIPOS_PORTAFOLIOS CAT ON CAT.TIPO = SM.TIPO_PORTAFOLIO
), MOVIMENTO_C3 AS (
	SELECT SM.*, CAT.DESCRIPCION VAR_MOVIMIENTO
    FROM MOVIMIENTO_C2 SM
    LEFT JOIN SIMS.SIMS_VARIANTES_MOVTOS CAT ON ( CAT.TIPO = SM.TIPO_MOVTO  AND CAT.CLAVE = SM.VARIANTE_MOVTO)
), TAB_FINAL AS (
	SELECT CLAVE,NOMBRE,FOLIO,TIPO_MOVTO,MOVIMIENTO, TIPO_PORTAFOLIO, PORTAFOLIO, VARIANTE_MOVTO, VAR_MOVIMIENTO, FECHA_OPERACION,FECHA_VALOR,NUMERO_EMISION,
	FECHA_CUPON_ACTUAL,TASA,DIAS,PRECIO,VALOR_NOMINAL,VALOR_NOMINAL_AMORTIZADO,VALOR_COSTO,INTERESES_ACUMULADOS,
	MONTO_MOVIMIENTO,MONTO_VRR,VALOR_NOMINAL_DISPONIBLE,INTERESES_ANTICIPADOS,INTERESES_PROVISIONADOS,INTERESES_COMPLEMENTO,
	UTILIDAD,PERDIDA,FOLIO_COMPRA,SUBFOLIO_COMPRA,ESTATUS_MOVTO,TIPO_CAMBIO_COMPRA,TIPO_CAMBIO_VENTA,TITULOS_VENCIMIENTO,
	FECHA_LIQUIDA,NUMERO_TITULOS,PRECIO_SUCIO,FECHA_LIQUIDACION
	FROM CLIENTES_ACTIVOS AC
	LEFT JOIN MOVIMENTO_C3 MV ON MV.CLAVE_CONTRAPARTE = AC.CLAVE
) SELECT * FROM TAB_FINAL
"""

In [6]:
path_dir_productos = get_table(path_dir_today, db_sims, "productos_clientes_sims", query = query_productos_sims)

# Transform the data and output the transformation

In [8]:
path_dir_output_transformation = os.path.join("/home/ale1726/proyects/datalake/clientes/data/productos/SIMS/agrupados", date_now)
os.makedirs(path_dir_output_transformation, exist_ok=True)
path_dir_output_transformation

'/home/ale1726/proyects/datalake/clientes/data/productos/SIMS/agrupados/21_04_2025'

In [9]:
productos_sims =  pd.read_csv(path_dir_productos, low_memory=False)

In [11]:
productos_sims["FECHA_OPERACION"] = pd.to_datetime(productos_sims["FECHA_OPERACION"])


In [39]:
productos_sims["AÑO_FOPE"] = productos_sims["FECHA_OPERACION"].dt.year.astype('Int64')
productos_sims["MES_FOPE"] = productos_sims["FECHA_OPERACION"].dt.month
productos_sims["MES_FOPE_STR"] = productos_sims["FECHA_OPERACION"].dt.month_name()        

#productos_meca["MES"] = productos_meca["OPE_FECHA_ALTA"].dt.monthv

In [31]:
productos_sims.columns

Index(['CLAVE', 'NOMBRE', 'FOLIO', 'TIPO_MOVTO', 'MOVIMIENTO',
       'TIPO_PORTAFOLIO', 'PORTAFOLIO', 'VARIANTE_MOVTO', 'VAR_MOVIMIENTO',
       'FECHA_OPERACION', 'FECHA_VALOR', 'NUMERO_EMISION',
       'FECHA_CUPON_ACTUAL', 'TASA', 'DIAS', 'PRECIO', 'VALOR_NOMINAL',
       'VALOR_NOMINAL_AMORTIZADO', 'VALOR_COSTO', 'INTERESES_ACUMULADOS',
       'MONTO_MOVIMIENTO', 'MONTO_VRR', 'VALOR_NOMINAL_DISPONIBLE',
       'INTERESES_ANTICIPADOS', 'INTERESES_PROVISIONADOS',
       'INTERESES_COMPLEMENTO', 'UTILIDAD', 'PERDIDA', 'FOLIO_COMPRA',
       'SUBFOLIO_COMPRA', 'ESTATUS_MOVTO', 'TIPO_CAMBIO_COMPRA',
       'TIPO_CAMBIO_VENTA', 'TITULOS_VENCIMIENTO', 'FECHA_LIQUIDA',
       'NUMERO_TITULOS', 'PRECIO_SUCIO', 'FECHA_LIQUIDACION', 'AÑO_FOPE',
       'MES_FOPE', 'MES_FOPE_STR'],
      dtype='object')

In [71]:
productos_sims.value_counts("NUMERO_TITULOS")

NUMERO_TITULOS
1000.0       212
2000.0       122
5000.0        99
10000.0       53
500.0         44
            ... 
3750.0         1
3938.0         1
4023.0         1
23.0           1
2500000.0      1
Name: count, Length: 61, dtype: int64

In [76]:
agrupado_test = (
    productos_sims.groupby(['AÑO_FOPE'], as_index=False)
    .agg(
        descripcion=('NUMERO_TITULOS', 'sum'), 
    ).sort_values(by=['AÑO_FOPE'], ascending=True)
)

In [77]:
agrupado_test

,AÑO_FOPE,descripcion
0,1992,0.0
1,1995,0.0
2,1996,0.0
3,1997,0.0
4,1998,0.0
5,1999,0.0
6,2000,0.0
7,2001,0.0
8,2002,0.0
9,2003,0.0


In [94]:
agrupado_portafolio = (
    productos_sims.groupby(['PORTAFOLIO'], as_index=False)
    .agg(
        valor=('FOLIO', 'count'), 
    ).sort_values(by=['PORTAFOLIO'], ascending=True)
)

In [95]:
agrupado_portafolio.to_csv(os.path.join(path_dir_output_transformation, "agrupado_portafolio.csv"), index=False)

In [89]:
agrupado_año_portafolio = (
    productos_sims.groupby(['AÑO_FOPE'], as_index=False)
    .agg(
        valor=('PORTAFOLIO', 'nunique'), 
    ).sort_values(by=['AÑO_FOPE'], ascending=True)
)

In [90]:
agrupado_año_portafolio

,AÑO_FOPE,valor
0,1992,1
1,1995,1
2,1996,2
3,1997,2
4,1998,2
5,1999,4
6,2000,4
7,2001,4
8,2002,4
9,2003,4


In [93]:
agrupado_año_portafolio.to_csv(os.path.join(path_dir_output_transformation, "agrupado_año_portafolio.csv"), index=False)

In [20]:
agrupado_operaciones_anuales = (
    productos_sims.groupby(['AÑO_FOPE'], as_index=False)
    .agg(
        num_operaciones=('FOLIO', 'count'), 
        monto_anuales_sum=('MONTO_MOVIMIENTO', 'sum'), 
        monto_anuales_min=('MONTO_MOVIMIENTO', 'min'), 
        monto_anuales_max=('MONTO_MOVIMIENTO', 'max'), 
        monto_anuales_mean=('MONTO_MOVIMIENTO', 'mean'), 
        monto_anuales_median=('MONTO_MOVIMIENTO', 'median')
    ).sort_values(by=['AÑO_FOPE'], ascending=True)
)

In [21]:
agrupado_operaciones_anuales.to_csv(os.path.join(path_dir_output_transformation, "agrupado_operaciones_anuales.csv"), index=False)

In [23]:
agrupado_operaciones_anuales_mes = (
    productos_sims.groupby(['AÑO_FOPE','MES_FOPE','MES_FOPE_STR'], as_index=False)
    .agg(
        num_operaciones=('FOLIO', 'count'), 
        monto_anuales_sum=('MONTO_MOVIMIENTO', 'sum'), 
        monto_anuales_min=('MONTO_MOVIMIENTO', 'min'), 
        monto_anuales_max=('MONTO_MOVIMIENTO', 'max'), 
        monto_anuales_mean=('MONTO_MOVIMIENTO', 'mean'), 
        monto_anuales_median=('MONTO_MOVIMIENTO', 'median')
    ).sort_values(by=['AÑO_FOPE','MES_FOPE'], ascending=True)
)

In [24]:
agrupado_operaciones_anuales_mes.to_csv(os.path.join(path_dir_output_transformation, "agrupado_operaciones_anuales_mes.csv"), index=False)

In [42]:
agrupado_plazo_anual = (
    productos_sims.groupby(['AÑO_FOPE'], as_index=False)
    .agg(
        plazo_mean=('DIAS', lambda x: round(x.mean())),
        plazo_min=('DIAS', 'min'), 
        plazo_max=('DIAS', 'max'),  
        plazo_median=('DIAS', 'median')
    ).sort_values(by=['AÑO_FOPE'], ascending=True)
)

In [44]:
agrupado_plazo_anual.to_csv(os.path.join(path_dir_output_transformation, "agrupado_plazo_anual.csv"), index=False)